In [69]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import pandas as pd
import time

In [70]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [71]:
driver = webdriver.Chrome('C:/Users/qudwn/OneDrive/바탕 화면/PythonWorkspace/chromedriver_win32/chromedriver.exe')

In [72]:
# 뮤지컬 <영웅> - 블루스퀘어
url = 'https://tickets.interpark.com/goods/23001653'
driver.get(url)

In [73]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [74]:
close = driver.find_element(By.XPATH, '//*[@id="popup-prdGuide"]/div/div[3]/button')
close.click()

In [75]:
review = driver.find_element(By.XPATH, '//*[@id="productMainBody"]/nav/div/div/ul/li[4]/a')
review.click()

In [76]:
def review_crawller(driver_):
    
    columns = ['rate', 'review_title', 'review_text', 'review_date']
    values = []
    soup = BeautifulSoup(driver_.page_source, 'lxml')

    data_rows = soup.find_all('li', attrs={'class':'bbsItem'}) # best review 중복 집계 막기 위해 처음 2개 제외, <영웅> 페이지에서는
                                                               # 해당되지 않음
    for row in data_rows:
        blank = []
        rate = row.find('div', attrs={'class':'prdStarIcon'})
        if rate:
            rate = rate['data-star'].strip()
            blank.append(rate)
        else:
            #print("<평점 없는 리뷰 제외합니다>")
            continue
        
        review_title = row.find('strong', attrs={'class':'bbsTitleText'})
        if review_title:
            review_title = review_title.get_text()
            blank.append(review_title)
        else:
            #print("<리뷰제목 없는 리뷰 제외합니다>")
            continue

        review_text = row.find('p', attrs={'class':'bbsText'})
        if review_text:
            review_text = review_text.get_text().strip()
            review_text = review_text.replace('\r', ' ') #줄바꿈 \r 표시 공백으로 변환
            blank.append(review_text)
        else:
            #print("<리뷰내용 없는 리뷰 제외합니다>")
            continue
        
        date = row.find_all('li', attrs={'class':'bbsItemInfoList'})[1] # 요소 순서대로 [예매자 id, 리뷰작성날짜, 조회수, 공감수]
        if date:
            date = date.get_text()
            blank.append(date)
        else:
            #print("<작성날짜 없는 리뷰 제외합니다>")
            continue
        
        values.append(blank)
        
    df = pd.DataFrame(values, columns = columns)
    df['musical_title'] = '영웅'
    df = df[['musical_title', 'rate', 'review_title', 'review_text', 'review_date']]
    
    return df

In [77]:
# df.review_text[1] # 짱 긴데 다 나왔음! 근데 문장 줄바꿈이 \r로 표시가 안된 텍스트는 문장끼리 이어지는 참사가..

In [78]:
soup = BeautifulSoup(driver.page_source, 'lxml')
num_page = int(soup.find('span', {'class': 'num'}).get_text())//15
num_last_list = int(soup.find('span', {'class': 'num'}).get_text())%15

In [79]:
print(num_page)
print(num_last_list)

62
10


In [80]:
review_list = []

# general case
for k in range((num_page//10)+1):
    try:
        # 첫 번째 페이지 크롤링
        print(k*10+1, '/', num_page)
        for j in range(15):
            more = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[3]/ul/li[{}]/div/div[2]/div[1]/a'.format(j+1)) # 따로 본인페이지에 맞게 Xpath 변경
            more.click() # //*[@id="prdReview"]/div/div[3]/ul/li[1]/div/div[2]/div[1]/a
            time.sleep(0.2)             
                         
        review_list.append(review_crawller(driver))

        # 2~10번째 페이지 크롤링
        for i in range(2, 11):
            print(k*10+i, '/', num_page)
            next_page = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[3]/div[2]/ol/li[{}]/a'.format(i))
            next_page.click()
            time.sleep(0.7)  # //*[@id="prdReview"]/div/div[3]/div[2]/ol/li[2]/a
            
            for j in range(15):
                try:                    
                    more = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[3]/ul/li[{}]/div/div[2]/div[1]/a'.format(j+1))
                    more.click()     # //*[@id="prdReview"]/div/div[3]/ul/li[1]/div/div[2]/div[1]/a
                    time.sleep(0.2)
                except:
                    pass
            
            review_list.append(review_crawller(driver))

        # 다음 열 개 페이지 로딩
        # 그런데 1-10에서 11-20으로 넘어가는 것만 좀 case가 달라서 따로 처리해줌
        if k == 0:
            next_ten = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[3]/div[2]/a')
            next_ten.click()   
            time.sleep(0.7)
        else:
            next_ten = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[3]/div[2]/a[2]')
            next_ten.click()   #//*[@id="prdReview"]/div/div[3]/div[2]/a[2]
            time.sleep(0.7)     
    except:                    
        pass


1 / 62
2 / 62
3 / 62
4 / 62
5 / 62
6 / 62
7 / 62
8 / 62
9 / 62
10 / 62
11 / 62
12 / 62
13 / 62
14 / 62
15 / 62
16 / 62
17 / 62
18 / 62
19 / 62
20 / 62
21 / 62
22 / 62
23 / 62
24 / 62
25 / 62
26 / 62
27 / 62
28 / 62
29 / 62
30 / 62
31 / 62
32 / 62
33 / 62
34 / 62
35 / 62
36 / 62
37 / 62
38 / 62
39 / 62
40 / 62
41 / 62
42 / 62
43 / 62
44 / 62
45 / 62
46 / 62
47 / 62
48 / 62
49 / 62
50 / 62
51 / 62
52 / 62
53 / 62
54 / 62
55 / 62
56 / 62
57 / 62
58 / 62
59 / 62
60 / 62
61 / 62
62 / 62
63 / 62
64 / 62


In [81]:
len(review_list)

63

In [82]:
df = pd.concat(review_list).reset_index(drop=True) # index 초기화 해줘야함
df.head()

,musical_title,rate,review_title,review_text,review_date
0,영웅,5,역시 영웅!,내인생 첫 뮤지컬이었는데 역시 영웅이네요! 웅장함과 감동에 너무 좋은 공연이었습니다!,2023.05.06
1,영웅,5,역시 영웅은 영웅!,보면서 전율이 몇번이나 흘렀는지 모르겠네요 ㅎㅎ 정재은님의 노래와 연기는 진짜.....,2023.05.06
2,영웅,5,최고의 선물!!,어린이날 선물로 선택했던 뮤지컬 영웅!마침 이벤트로 부르마불 선물주셔서 감사했어요!...,2023.05.06
3,영웅,5,영웅 관람후기,제대로 된 뮤지컬을 처음 관람한거나 마찬가지인데현장의 그 여운과 생생함이 아직도 안...,2023.05.06
4,영웅,5,그냥 최고의 뮤지컬,"안 본 눈 삽니다,,",2023.05.06


In [83]:
df.tail()

,musical_title,rate,review_title,review_text,review_date
936,영웅,5,"밍중근 최고, but 관객매너 망..",17일 공연에서 민우혁 배우님 처음으로 봤는데 정말 안중근 그 자체였습니다. 마곡에...,2023.03.18
937,영웅,4,재미 있습니다,"오랜만에 즐겁고 , 활기찬 시간이였습니다",2023.03.18
938,영웅,5,민우혁 배우,민우혁 배우의 재발견이였습니다. 눈물이 멈추질 않았네요. 감사합니다.,2023.03.18
939,영웅,1,기대하던 영웅을 접하고 실망이 이만저만 아닙니다,일단 설희라는 캐릭터가 왜 필요한지 모르겠습니다어떠한 이유인지 몰라도 가상의 인물을...,2023.03.18
940,영웅,5,첫공 갔다왔어요,"두번째 관람이라 놀라움은 없지만 그래도 가끔씩 닭살 돋네요, 언젠지 모르겠지만 또 ...",2023.03.18


In [85]:
df.to_csv('영웅_크롤링.csv', encoding = 'utf-8-sig')

In [87]:
data = pd.read_csv('C:/Users/qudwn/OneDrive/바탕 화면/DA주니어프로젝트_뮤지컬/크롤링 코드/영웅_크롤링.csv')
data = data.iloc[:, 1:]
data.head()

,musical_title,rate,review_title,review_text,review_date
0,영웅,5,역시 영웅!,내인생 첫 뮤지컬이었는데 역시 영웅이네요! 웅장함과 감동에 너무 좋은 공연이었습니다!,2023.05.06
1,영웅,5,역시 영웅은 영웅!,보면서 전율이 몇번이나 흘렀는지 모르겠네요 ㅎㅎ 정재은님의 노래와 연기는 진짜.....,2023.05.06
2,영웅,5,최고의 선물!!,어린이날 선물로 선택했던 뮤지컬 영웅!마침 이벤트로 부르마불 선물주셔서 감사했어요!...,2023.05.06
3,영웅,5,영웅 관람후기,제대로 된 뮤지컬을 처음 관람한거나 마찬가지인데현장의 그 여운과 생생함이 아직도 안...,2023.05.06
4,영웅,5,그냥 최고의 뮤지컬,"안 본 눈 삽니다,,",2023.05.06


In [27]:
df.to_excel('뮤지컬_영웅_크롤링.xlsx', encoding = 'utf-8-sig')

In [30]:
data = pd.read_excel('C:/Users/qudwn/OneDrive/바탕 화면/DA주니어프로젝트_뮤지컬/크롤링 코드/뮤지컬_영웅_크롤링.xlsx')
data = data.iloc[:, 1:]
data.head()

,musical_title,rate,review_title,review_text,review_date
0,영웅,5,역시 영웅은 영웅!,보면서 전율이 몇번이나 흘렀는지 모르겠네요 ㅎㅎ 정재은님의 노래와 연기는 진짜.....,2023.05.06
1,영웅,5,최고의 선물!!,어린이날 선물로 선택했던 뮤지컬 영웅!마침 이벤트로 부르마불 선물주셔서 감사했어요!...,2023.05.06
2,영웅,5,영웅 관람후기,제대로 된 뮤지컬을 처음 관람한거나 마찬가지인데현장의 그 여운과 생생함이 아직도 안...,2023.05.06
3,영웅,5,그냥 최고의 뮤지컬,"안 본 눈 삽니다,,",2023.05.06
4,영웅,5,어린이날 관람,거의 전석이 꽉 차고 어린이날이라 어린이 관람객이 많아서 혹시나 걱정했는데.. 뮤지...,2023.05.06
